# landscraper - doing the dirty work for intellectual property (IP) decisions

__Contributer: Akhil Jindal__ | https://github.com/akhil-jindal/

### Library Imports:

In [1]:
from bs4 import BeautifulSoup
import requests
import os
import csv
import pandas as pd
import re
from glob import glob

### Classes and functions:

In [2]:
class Patent:
    
    def __init__(self, title, number, assignee, pub_date, source):
        """Defining pertinent attributes
        """
        self.title = title
        self.number = number
        self.assignee = assignee
        self.pub_date = pub_date
        self.source = source
           
    def soupify(self):
        """Soupify for BeautifulSoup
        """
        src = requests.get('{}'.format(self.source)).text
        soup = BeautifulSoup(src, 'lxml')
        return soup
    
    def description(self, soup):
        """Returns a detailed description of target patent application
        Can include Abstract, Summary and Background
        """
        description = soup.find('div', class_='description').text
        return description
      
    def genContent(self, classification, corpus_dir, soup):
        """Generates a training document for a corpus
        """
        class_dir = os.path.join(corpus_dir, classification)
        file_name = re.sub('[^A-Za-z0-9]+', '', self.number)
        
        if os.path.isdir(class_dir):
            write_path = os.path.join(class_dir, file_name)
            
            try:
                self.writeContent(write_path)
                
            except:
                 print ("Failed to generate {}".format(self.number))
        
        else:
            os.mkdir(class_dir)
            write_path = os.path.join(class_dir, file_name)
            
            try:
                self.writeContent(write_path)

            except:
                 print ("Failed to generate {}".format(self.number))
        
    
    def writeContent(self, write_file):
        """Writes the content for a training document
        """
        with open(write_file, "w") as wf:
            header = str(self.title) + "\n" + str(self.number) + "\n" + str(self.assignee) + "\n" + str(self.pub_date) + "\n" + str(self.source) + "\n"
            wf.write(header)
            try:
                desc = self.description(soup).encode('utf-8').strip()
                wf.write(desc)
            except:
                print ("Encoding messed up for {}".format(write_file))
        wf.close()

In [3]:
def cleanCSV(csv_file):
    """The .csv file downloadable from patents.google.com.
    However, for our purposes, the downloaded .csv has more information than we need.
    We keep pertient columns, and delete rows that are associated with non-US patents.
    """
    
    df = pd.read_csv(csv_file, error_bad_lines=False, sep=',')
    df = df.drop(["inventor/author", "priority date", "filing/creation date", 
                  "grant date", "representative figure link"], axis = 1)
    df = df[df["id"].str.contains("US")]
    return df

def definePatent(dataframe):
    """Uses Patent class to generate patents for further processing
    """
    patents = []
    for index, row in dataframe.iterrows():
        patent = Patent(row["title"], row["id"], row["assignee"], row["publication date"], row["result link"])
        patents.append(patent)
    return patents

def classify(csv_file):
    classification = os.path.basename(csv_file).split(".")[0]
    return classification

### Preparing Corpus:

Note:

* The .csv's which contain identification information for each Patent were manually downloaded from patents.google.com.

* Certain certain classifications will yield more patents than others.  Down sampling may be necessary to ensure a balanced data set for training. 

__Inputs:__

In [4]:
csv_dir = "/home/ajindal/Documents/landscraper/class_csv/"
corpus_dir = "/home/ajindal/Documents/landscraper/corpus"

__Getting Training Documents__

In [5]:
globabble_dir = os.path.join(csv_dir, "*")

for csv_file in glob(globabble_dir):
    classification = classify(csv_file)
    
    print("Preparing the following class: {}".format(classification))
    clean = cleanCSV(csv_file)
   
    patents = definePatent(clean)
          
    for patent in patents:
        soup = patent.soupify()
        patent.genContent(classification, corpus_dir, soup)

Preparing the following class: G06K
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2008166026A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2005058350A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2008296393A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2010014758A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2011286628A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2010054585A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2007172099A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2005281449A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2009048842A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2016155011A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2012138674A1
Encoding messed up

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2007103729A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2009087243A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2015339525A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2014193029A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2006265452A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2009245658A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2005163352A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2013044942A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2013094695A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2007230793A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06K/US2014270384A1
Encoding messed up for /home/ajindal/Documents/landscr

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2003033205A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2004073507A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2015310468A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2012066054A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2005125251A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2002042752A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2009119168A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2006085259A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2005246240A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2005055299A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2013036063A1
Encoding messed up for /home/ajindal/Documents/landscr

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2006085239A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2013006805A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2009119159A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2011154451A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2002010648A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2002111914A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2001034628A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2004158491A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2008059314A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2012310838A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2007198407A1
Encoding messed up for /home/ajindal/Documents/landscr

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2014207550A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2019043132A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2009132344A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2013060595A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2002198926A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2011246279A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2016371733A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2011276385A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2002138396A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2018349988A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06Q/US2016092993A1
Encoding messed up for /home/ajindal/Documents/landscr

...

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2014282228A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2014282477A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2017124191A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2014085206A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2009172705A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2015067546A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2012278515A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2016277540A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2013166852A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2008320459A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2013154957A1
Encoding messed up for /home/ajindal/Documents/landscr

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2019087030A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2013162567A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2013282676A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2011248946A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2009013017A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2017031631A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2008033949A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2019095129A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2016065690A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2006020701A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2016179789A1
Encoding messed up for /home/ajindal/Documents/landscr

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2015193526A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2014046917A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2012047124A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2017115999A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018314568A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2015052396A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018357133A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2015278190A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018196831A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2017060892A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2019044928A1
Encoding messed up for /home/ajindal/Documents/landscr

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2017277379A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2007239727A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2009235213A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018300466A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2009295751A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018373869A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2011271064A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2014258720A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018314441A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2016379776A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2008281992A1
Encoding messed up for /home/ajindal/Documents/landscr

Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2015116208A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2016070467A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2002161991A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2008189525A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2019097801A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018165173A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2015026552A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018095786A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2018292943A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2017212879A1
Encoding messed up for /home/ajindal/Documents/landscraper/corpus/G06F/US2014232666A1
Encoding messed up for /home/ajindal/Documents/landscr